In [1]:
!wc -l /mnt/slow_drive/dmytro/google-en-1M-3-grams/googlebooks-eng-1M-3gram-20090715-100.csv

52992736 /mnt/slow_drive/dmytro/google-en-1M-3-grams/googlebooks-eng-1M-3gram-20090715-100.csv


In [2]:
!head /mnt/slow_drive/dmytro/google-en-1M-3-grams/googlebooks-eng-1M-3gram-20090715-100.csv

"! "" Amongst"	1801	3	3	3
"! "" Amongst"	1803	1	1	1
"! "" Amongst"	1809	2	2	2
"! "" Amongst"	1811	1	1	1
"! "" Amongst"	1812	2	2	2
"! "" Amongst"	1815	2	2	2
"! "" Amongst"	1818	1	1	1
"! "" Amongst"	1820	2	2	2
"! "" Amongst"	1821	3	3	3
"! "" Amongst"	1822	3	3	3


In [44]:
import csv
import json
import sys

from sklearn.metrics import classification_report

print(sys.platform, sys.maxsize)
csv.field_size_limit(sys.maxsize)
print('After:', csv.field_size_limit())

linux 9223372036854775807
After: 9223372036854775807


In [4]:
import pandas as pd
import spacy

data_dir = '/mnt/slow_drive/dmytro/google-en-1M-3-grams/'
train_filename = data_dir + 'googlebooks-eng-1M-3gram-20090715-100.csv'


In [5]:
nlp = spacy.load("en_core_web_md")

In [10]:
def read_data(filename):
    rows = []
    with open(filename) as tsvfile:
        reader = csv.reader(tsvfile, delimiter='\t')
        prev_row = None
        for row in reader:
            if len(row) == 1:
                row = row[0].strip().split('\t')
            if row[1] < '2000':
                continue

            if prev_row and row[0] == prev_row[0]:
                prev_row[1] += int(row[2])
            else:
                try:
                    prev_row = [row[0], int(row[2])]
                except:
                    print(row)
                    raise
                rows.append(prev_row)

    return rows

all_rows = []
for i in range(1, 200):
    train_filename = data_dir + 'googlebooks-eng-1M-3gram-20090715-{}.csv'.format(i)
    all_rows.extend(read_data(train_filename))
    print('Filename {} is read, current size: {}'.format(train_filename, len(all_rows)))

Filename /mnt/slow_drive/dmytro/google-en-1M-3-grams/googlebooks-eng-1M-3gram-20090715-1.csv is read, current size: 518408
Filename /mnt/slow_drive/dmytro/google-en-1M-3-grams/googlebooks-eng-1M-3gram-20090715-2.csv is read, current size: 1038038
Filename /mnt/slow_drive/dmytro/google-en-1M-3-grams/googlebooks-eng-1M-3gram-20090715-3.csv is read, current size: 1559405
Filename /mnt/slow_drive/dmytro/google-en-1M-3-grams/googlebooks-eng-1M-3gram-20090715-4.csv is read, current size: 2078597
Filename /mnt/slow_drive/dmytro/google-en-1M-3-grams/googlebooks-eng-1M-3gram-20090715-5.csv is read, current size: 2598599
Filename /mnt/slow_drive/dmytro/google-en-1M-3-grams/googlebooks-eng-1M-3gram-20090715-6.csv is read, current size: 3117882
Filename /mnt/slow_drive/dmytro/google-en-1M-3-grams/googlebooks-eng-1M-3gram-20090715-7.csv is read, current size: 3638166
Filename /mnt/slow_drive/dmytro/google-en-1M-3-grams/googlebooks-eng-1M-3gram-20090715-8.csv is read, current size: 4158313
Filename 

In [12]:
tokens_freq = []
for row in all_rows:
    if row[1] < 2:
        continue
    if '\t' in row[0]:
        continue
    cur_tokens = row[0].split()
    if len(cur_tokens) == 3:
        tokens_freq.append((cur_tokens[0], cur_tokens[1], cur_tokens[2], row[1]))

In [98]:
df = pd.DataFrame(tokens_freq)

In [99]:
df.columns = ['first', 'second', 'third', 'count']
df

,first,second,third,count
0,!,"""",148,14
1,!,"""",Autumn,19
2,!,"""",Booth,5
3,!,"""",Chan,9
4,!,"""",Hiram,10
...,...,...,...,...
52858392,•,when,their,6
52858393,•,with,truth,3
52858394,•,within,-,5
52858395,•,would,the,5


In [100]:
df.to_csv('/mnt/slow_drive/dmytro/google-1M-filtered-3grams.csv', index=False)

In [101]:
df[(df['first'] == 'in') & (df['third'] == 'meantime')]

,first,second,third,count
19783403,in,this,meantime,3
26437514,in,value,meantime,2
26969689,in,the,meantime,15654
43686899,in,die,meantime,8


Enough of 3-grams for now. Let's build ...
# ... solution

In [36]:
test_filename = '../../../tasks/06-language-as-sequence/run-on-test.json'
with open(test_filename) as f:
    test_data = json.load(f)

test_tokens = []
test_classes = []
for sentence in test_data:
    for word in sentence:
        test_tokens.append(word[0])
        test_classes.append(word[1])

In [41]:
' '.join(test_tokens[:20])

'I think the magnitude of a benefit and error rates that were chosen were reasonable They were standard from our'

In [43]:
' '.join([str(c) for c in test_classes[:20]])

'False False False False False False False False False False False False False False True False False False False False'

# Baseline FTW
let's just predict everything as `False`

In [46]:
print(classification_report(test_classes, [False] * len(test_tokens)))

              precision    recall  f1-score   support

       False       0.97      1.00      0.98      4542
        True       0.00      0.00      0.00       155

    accuracy                           0.97      4697
   macro avg       0.48      0.50      0.49      4697
weighted avg       0.94      0.97      0.95      4697



# Let's take it further

In [ ]:
trigrams = {}

def add_trigram(dest, tokens, freq):
    if tokens[0] not in dest:
        dest[tokens[0]] = {}
    if tokens[1] not in dest[tokens[0]]:
        dest[tokens[0]][tokens[1]] = {}
    if tokens[2] not in dest[tokens[0]][tokens[1]]:
        dest[tokens[0]][tokens[1]][tokens[2]] = freq

    dest[tokens[0]][tokens[1]][tokens[2]] += freq

for index, row in df.iterrows():
    add_trigram(trigrams, row[:3], row[3])

In [113]:
print(len(trigrams))
trigrams.get('in', {}).get('the', {}).get('meantime')

262682


31308

In [105]:
def predict_by_ngram(prev, cur, next_):
    end_query = 'first == "{first}" & second == "{second}" & third in (".", "?", "!")'.format(first=prev, second=cur)
    print(end_query)
    not_end_query = 'first == "{first}" & second == "{second}" & third == "{third}"'.format(
        first=prev, second=cur, third=next_
    )
    print(not_end_query)
    end_df = df.query(end_query)
    not_end_df = df.query(not_end_query)
    print(end_df)
    print(not_end_df)


prev_token = ''
next_token = ''

for i, token in enumerate(test_tokens):
    if i < len(test_tokens) - 1:
        next_token = test_tokens[i + 1]
    print('|{}| -> |{}| -> |{}|'.format(prev_token, token, next_token))
    predict_by_ngram(prev_token, token, next_token)
    prev_token = token

    print('=' * 10, i, '=' * 10)
    #prev = token
    if i > 15:
        break

|| -> |I| -> |think|
first == "" & second == "I" & third in (".", "?", "!")
first == "" & second == "I" & third == "think"
Empty DataFrame
Columns: [first, second, third, count]
Index: []
Empty DataFrame
Columns: [first, second, third, count]
Index: []
========== 0 ==========
|I| -> |think| -> |the|
first == "I" & second == "think" & third in (".", "?", "!")
first == "I" & second == "think" & third == "the"
Empty DataFrame
Columns: [first, second, third, count]
Index: []
         first second third  count
43317953     I  think   the  27524
========== 1 ==========
|think| -> |the| -> |magnitude|
first == "think" & second == "the" & third in (".", "?", "!")
first == "think" & second == "the" & third == "magnitude"
          first second third  count
25462201  think    the     .     10
Empty DataFrame
Columns: [first, second, third, count]
Index: []
========== 2 ==========
|the| -> |magnitude| -> |of|
first == "the" & second == "magnitude" & third in (".", "?", "!")
first == "the" & secon